In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from glob import glob

In [2]:
files = glob('url/*.json')
len(files)

548

In [3]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        urls.extend(json.load(fopen))

100%|██████████████████████████████████████| 548/548 [00:00<00:00, 15074.07it/s]


In [4]:
# !rm -rf page
!mkdir page

In [5]:
urls = sorted(list(set(urls)))
len(urls)

6454

In [6]:
urls[0]

'https://dewanbahasa.jendeladbp.my/2021/02/19/22/'

In [8]:
r = requests.get(urls[0])
soup = BeautifulSoup(r.content, "lxml")

In [13]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        title = soup.find('h1', {'class': 'post-header-title'}).text.strip()
    except:
        title = None
    try:
        p = soup.find('div', {'class': 'post-body-content'}).find_all('p')
        p = [p_.text for p_ in p]
    except:
        p = None
    
    data = {
        'url': url,
        'title': title,
        'p': p,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    aranged = np.arange(i, i + max_worker)
    urls_ = [(urls[a], os.path.join('page', f'{a}.json')) for a in aranged]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

    for future in as_completed(futures):
        future.result()

  0%|                                                   | 0/646 [00:00<?, ?it/s]